In [ ]:
from pathlib import Path
import csv
import subprocess

experiments_dir = Path().resolve()
results_dir = experiments_dir / "results" / "results"

trials_folders = []
batch_folders = []
n_gens=5000

for subfolder in results_dir.iterdir():
    if subfolder.is_dir():
        batch_folders.append(Path(subfolder))

for b_folder in batch_folders:

    for t_folder in b_folder.iterdir():

        if b_folder.name == "completed":
            continue

        print(f"BATCH: {b_folder.name}")

        max_gen = 0
        min_gen = 10000
        min_gen_file = ""
        trials_done = []
        trials_status = []
        trials_to_rerun= []

        for subfolder in t_folder.iterdir():
            if subfolder.is_dir():
                fitness_file = subfolder / "fitness.csv"
                with open(fitness_file, mode='r') as file:
                    csv_reader = csv.DictReader(file)
                    rows = list(csv_reader)
                    last_row = rows[-1] if rows else None
                    if last_row:
                        gen = int(last_row["gen"])
                        folder_name = subfolder.resolve().name

                        if min_gen > gen:
                            min_gen = gen
                            min_gen_file = folder_name
                        if max_gen < gen:
                            max_gen=gen
                        
                        if gen == n_gens:
                            trials_done.append(int(subfolder.resolve().name.split("_")[1]))
                        
                        trial_name = "_".join((*folder_name.split("_")[1:], t_folder.name, b_folder.name))
                        result = subprocess.run(
                        ["squ", f"--name={trial_name}", "--format='%.2t'"],
                        capture_output=True, text=True)
                        trials_status.append((int(folder_name.split("_")[1]), result.stdout[6:8].strip()))
        
        trials_done.sort()
        print(f"MAX_GEN: {max_gen}")
        trial_name = "_".join((*min_gen_file.split("_")[1:], t_folder.name, b_folder.name))
        print(f"MIN_GEN: {min_gen} {trial_name}")
        print(f"TRIALS DONE: {trials_done}")
        print(f"TRIALS STATUS: {trials_status}")
        for status in trials_status:
            if status[0] not in trials_done:
                if status[1] == 'R' or status[1] == 'PD':
                    continue
                else:
                    trials_to_rerun.append(status[0])
        print(f"TRIALS TO RERUN: {trials_to_rerun}\n")
            
            